In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [37]:
data= pd.read_csv('../input/seattleWeather_1948-2017.csv')
data=data.drop(["PRCP"],axis=1)

data.head()

In [38]:
pd.options.display.float_format = '{:.2f}%'.format ## Set a percentage view as a default for Float
def missing_values(n):
    df=pd.DataFrame()
    df["missing, %"]=data.isnull().sum()*100/len(data.isnull())
    df["missing, num"]=data.isnull().sum()
    return df.sort_values(by="missing, %", ascending=False)
missing_values(data)

In [5]:
data.dropna(inplace=True)

In [6]:
data.isnull().sum()

In [7]:
missing_values(data)

Dealing with missing values for the machine learning : https://towardsdatascience.com/7-ways-to-handle-missing-values-in-machine-learning-1a6326adf79e

In [39]:
x=data.iloc[:,1:-1].values
x

In [40]:
y=data.iloc[:,-1].values
y

## Encoding of the Rain prediction column

In [41]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(y)

In [12]:
y

## Split the dataset into Training set and Test Set

In [42]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, random_state=0)

In [43]:
print(x_train)

In [44]:
print(y_train)

## Feature Scaling 

In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train[:]=sc.fit_transform(x_train[:])
x_test[:] = sc.transform(x_test[:])

In [17]:
x_train

In [18]:
x_test

The main difference between Regression and Classification algorithms that Regression algorithms are used to predict the continuous values such as price, salary, age, etc. and Classification algorithms are used to predict/Classify the discrete values such as Male or Female, True or False, Spam or Not Spam, etc. (https://www.javatpoint.com/regression-vs-classification-in-machine-learning#:~:text=The%20main%20difference%20between%20Regression,Spam%20or%20Not%20Spam%2C%20etc.) 


# GradientBoostingClassifier 

Gradient Boosting ensemble is an ensemble created from decision trees added sequentially to the model. Gradient boosting refers to a class of ensemble machine learning algorithms that can be used for classification or regression predictive modeling problems.

There are three types of enhancements to basic gradient boosting that can improve performance:

- Tree Constraints: such as the depth of the trees and the number of trees used in the ensemble.
- Weighted Updates: such as a learning rate used to limit how much each tree contributes to the ensemble.
- Random sampling: such as fitting trees on random subsets of features and samples
https://machinelearningmastery.com/gradient-boosting-machine-ensemble-in-python/

## Gradient Boosting for Classification


In [19]:
# evaluate gradient boosting algorithm for classification
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
#define the model
model = GradientBoostingClassifier()
# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1) 


n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
#the mean and standard deviation of the accuracy of the model across all repeats and folds.

In [20]:
n_scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
#The mean absolute error (MAE) larger negative MAE are better


Cross-validator (Cross-validation is a resampling procedure used to evaluate machine learning models on a limited data sample.)
(https://machinelearningmastery.com/k-fold-cross-validation/#:~:text=Cross%2Dvalidation%20is%20primarily%20used,the%20training%20of%20the%20model.)
n-slips:Number of folds. Must be at least
2. n_repeats - Number of times cross-validator needs to be repeated.
3. random_state - Controls the generation of the random states for each repetition. Pass an int for reproducible output across multiple function calls
 evaluate the model on the dataset

The mean absolute error (MAE) of the model across all repeats and folds. The scikit-learn library makes the MAE negative so that it is maximized instead of minimized. This means that larger negative MAE are better and a perfect model has a MAE of 0. Mean Absolute Error is a model evaluation metric used with regression models. The mean absolute error of a model with respect to a test set is the mean of the absolute values of the individual prediction errors on over all instances in the test set. Each prediction error is the difference between the true value and the predicted value for the instance. (https://link.springer.com/referenceworkentry/10.1007/978-0-387-30164-8_525)

In [ ]:
 # make predictions using gradient boosting for classification
model = GradientBoostingClassifier()
#fit the model on the whole dataset
model.fit(x,y)
#make a single prediction
row=[53,46]
yhat=model.predict([row])
print('Prediction: %d' % yhat[0])

 https://machinelearningmastery.com/gradient-boosting-machine-ensemble-in-python/

## Gradient Boosting for Regression


The idea of gradient boosting is to improve weak learners and create a final combined prediction model. Decision trees are mainly used as base learners in this algorithm. The weak learner is identified by the gradient in the loss function. The prediction of a weak learner is compared to actual value and error is calculated. Based on this error, the model can determine the gradient and change the parameters to decrease the error rate in the next training. (https://www.datatechnotes.com/2019/06/gradient-boosting-regression-example-in.html)

In [23]:
#data=data.drop(["PRCP"],axis=1)

# 2 Converting object into datetime to extract day, month and year
from datetime import datetime
data["DATE"]=pd.to_datetime(data["DATE"], format= "%Y-%m-%d")

# Extract day, month and year
data["DAY"]=data["DATE"].dt.day
data["MONTH"]=data["DATE"].dt.month
data["YEAR"]=data["DATE"].dt.year
data=data.drop(["DATE"], axis=1)

#Rearrange columns
data=data[["DAY", "MONTH", "YEAR", "TMAX", "TMIN", "RAIN"]]
data.head()


## Make a prediction of TMAX on a particular day


In [24]:
x=data.iloc[:,:-3].values
y= data.iloc[:,-3].values

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, random_state=0)

In [26]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train[:]=sc.fit_transform(x_train[:])
x_test[:] = sc.transform(x_test[:])

In [27]:
#x_train
#y_test

In [28]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RepeatedKFold


# define the model
model = GradientBoostingRegressor()
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model
n_scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

### Difference between K-Fold and Stratified CV
KFold is a cross-validator that divides the dataset into k folds. Stratified is to ensure that each fold of dataset has the same proportion of observations with a given label.

## Predict


In [29]:
# define the model
model = GradientBoostingRegressor()
# fit the model on the whole dataset
model.fit(x, y)
# make a single prediction
row=[1,1,2022]
yhat = model.predict([row])
# summarize prediction
print('Prediction: %d' % yhat[0])

In [30]:
#x_ax = range(len(y_test))
#plt.scatter(x_ax, y_test, s=5, color="blue", label="original")
#plt.plot(x_ax, yhat, lw=0.8, color="red", label="predicted")
#plt.legend()
#plt.show()


## Gradient Boosting Hyperparameters


In [31]:
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

## Number of decision trees in the ensemble is the important hyperparameter of the Gradient Boosting (n_estimators)
decision trees are added to the model sequentially in an effort to correct and improve upon the predictions made by prior trees. As such, more trees is often better. 


The number of trees must also be balanced with the learning rate, e.g. more trees may require a smaller learning rate, fewer trees may require a larger learning rate.

## Learning Rate and Logarithmic loss

In [49]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib
from matplotlib import pyplot
matplotlib.use('Agg')


In [53]:
data= pd.read_csv('../input/seattleWeather_1948-2017.csv')
data=data.drop(["PRCP"],axis=1)
data.dropna(inplace=True)
x=data.iloc[:,1:-1]
y=data.iloc[:,-1] 
label_encoded_y = LabelEncoder().fit_transform(y)


We can use the grid search capability in scikit-learn to evaluate the effect on logarithmic loss of training a gradient boosting model with different learning rate values.



In [58]:
model = XGBClassifier()
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learning_rate=learning_rate)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(x, label_encoded_y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
   


learning_rate = 0.1 is the best result --> the default number of trees is perfect for this model. We don't need to increase the number of trees

Smaller learning rates generally require more trees to be added to the model. An interesting link to the YouTube that explains the Gradient Boosting: https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/



## Learning Rate & Number of Trees

In [60]:
model = XGBClassifier()
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
n_estimators=[100,200,300,400,500]
param_grid = dict(learning_rate=learning_rate, n_estimators = n_estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(x, label_encoded_y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

As we can see the most optimal is n_estimator = 100 with the learning_rate =0.1 (which is by default)

## Number of Trees 

In [69]:
model = GradientBoostingClassifier()
n_estimators=[10,50,100,500,1000,5000]
param_grid = dict(n_estimators = n_estimators)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, x, label_encoded_y, scoring='accuracy', cv=cv, n_jobs=-1)
grid_result = grid_search.fit(x, label_encoded_y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

 - param_griddict or list of dictionaries. Dictionary with parameters names ( str ) as keys and lists of parameter settings to try as values, or a list of such dictionaries, in which case the grids spanned by each dictionary in the list are explored. This enables searching over any sequence of parameter settings (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#:~:text=param_griddict%20or%20list%20of,any%20sequence%20of%20parameter%20settings.) 
 - RepeatedStratifiedKFold --> 
     - n_splits - number of folds  - 10 folds (fold --> the folds are made by preserving the percentage of samples for each class)
     - n_repeats - number of times cross-validator needs to be repeated - 3 times
     - random_state - controls the generation of the random states for each repeatition.
 - cross_val_score - evaluates a score by cross-validation 
     - model (Gradient Boosting)
     - x, y (label encoded (1,0)
     - scoring - in our case we use the accuracy scoring
     - cv (cross-validation indicator)
     - n_jobs - number of jobs to run in parallel. -1 means using all jobs
 - grid_search - Exhaustive search over specified parameter values for an estimator.

 - The grid.best_score_ is the average of all cv folds for a single combination of the parameters you specify in the tuned_params.
 - grid_result.cv_results_["mean_test_score"] - The mean_test_score that sklearn returns is the mean calculated on all samples where each sample has the same weight.





## Sub-sample 
Using fewer samples introduces more variance for each tree, although it can improve the overall performance of the model. 
The number of samples used to fit each tree is specified by the “subsample” argument and can be set to a fraction of the training dataset size. By default, it is set to 1.0 to use the entire training dataset.

(https://machinelearningmastery.com/gradient-boosting-machine-ensemble-in-python/)




In [75]:
from numpy import mean
from numpy import std
from numpy import arange
# get a list of models to evaluate
def get_models():
	models = dict()
	# explore sample ratio from 10% to 100% in 10% increments
	for i in arange(0.1, 1.1, 0.1):
		key = '%.1f' % i
		models[key] = GradientBoostingClassifier(subsample=i)
	return models
 
# evaluate a given model using cross-validation
def evaluate_model(model, x, label_encoded_y):
	# define the evaluation procedure
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# evaluate the model and collect the results
	scores = cross_val_score(model, x, label_encoded_y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores
 
# define dataset
 # get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	# evaluate the model
	scores = evaluate_model(model, x, label_encoded_y)
	# store the results
	results.append(scores)
	names.append(name)
	# summarize the performance along the way
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

In [78]:
from matplotlib import pyplot

# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

# Number of Features (max_features)
The number of features used to fit each decision tree can be varied.

Like changing the number of samples, changing the number of features introduces additional variance into the model, which may improve performance, although it might require an increase in the number of trees.

 defaults to all features in the training dataset.

In [82]:
from numpy import mean
from numpy import std
from numpy import arange
from sklearn.datasets import make_classification

# get the dataset
def get_dataset():
	X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)
	return X, y

# get a list of models to evaluate
def get_models():
	models = dict()
	# explore number of features from 1 to 20
	for i in range(1,21):
		models[str(i)] = GradientBoostingClassifier(max_features=i)
	return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
	# define the evaluation procedure
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# evaluate the model and collect the results
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores

# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	# evaluate the model
	scores = evaluate_model(model, X, y)
	# store the results
	results.append(scores)
	names.append(name)
	# summarize the performance along the way
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison


## Grid Search Hyperparameters (n_estimators, learning_rate, subsample, max_depth)

In [ ]:
model = GradientBoostingClassifier()
grid=dict()
grid['n_estimators'] = [10, 50, 100, 500]
grid['learning_rate'] = [0.0001, 0.001, 0.01, 0.1, 1.0]
grid['subsample'] = [0.5, 0.7, 1.0]
grid['max_depth'] = [3, 7, 9]

#evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
#define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid,  cv=cv, scoring='accuracy',n_jobs=-1)
#execute the grid search procedure
grid_result = grid_search.fit(x, label_encoded_y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))